# Import

In [1]:
import csv
import os
import pickle

import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import adjusted_mutual_info_score

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [2]:
tqdm.pandas()

# Read data

In [3]:
ag_news_df = pd.read_csv("../../../Preprocessing/data/AgNews/master.csv", index_col=0)

In [4]:
with open("../../../Preprocessing/data/AgNews/class.csv", mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

In [5]:
vector_sizes = [2, 3, 4, 6, 7, 8, 9, 10, 20, 40, 80, 160, 320, 640]

In [6]:
label = ag_news_df["class"].to_numpy()

In [7]:
model_nums = 2
vector_model_num = 0

In [12]:
def kmeans_value(gmm, vectors, label):
    pred = gmm.predict(vectors)
    mi = adjusted_mutual_info_score(pred, label)
    return {"pred": pred, "mi":mi}

In [13]:
vectors_path = "../../../Clustering/data/AgNews/doc2vec/vector/"
models_path = "../../../Clustering/data/AgNews/doc2vec/KMeans/model/"
pred_path = "../../../Clustering/data/AgNews/doc2vec/KMeans/pred/"
stats_list = []
for model_num in range(model_nums):
    stats = []
    for vector_size in tqdm(vector_sizes):
        vector_path = f"{vectors_path}{vector_size}/normalized/{vector_model_num}.csv"
        vectors = np.loadtxt(vector_path, delimiter=",")
        
        kmeans = pickle.load(open(f"{models_path}{vector_size}/{model_num}.sav", "rb"))
        value = kmeans_value(kmeans, vectors, label)
        pred, *stat = value.items()
        stats.append(dict(stat))
    stats_list.append(stats)

100%|██████████| 14/14 [16:02<00:00, 68.75s/it] 


In [14]:
stats_list = [pd.DataFrame(stats, index=vector_sizes) for stats in stats_list]

In [15]:
stats_path = "../../data/AgNews/doc2vec/KMeans/stats/"
for model_num in range(model_nums):
    os.makedirs(f"{stats_path}", exist_ok=True)
    stats_list[model_num].to_csv(f"{stats_path}{model_num}.csv")

In [16]:
stats_list[0]

,mi
2,0.225923
3,0.288683
4,0.395076
6,0.455027
7,0.464722
8,0.456285
9,0.451079
10,0.440101
20,0.267159
40,0.019844
